ID Number: 33385806

# EEG-based BCI using Emotiv Epoc X and Visual Imagery: An Exploratory Study

## MSc Project for Computational Cognitive Neuroscience 2020/2021

1. Explore the spectral content through standard periodogram and Welch's method
2. Compute Time-Frequency Analysis using Morlet Wavelet Transform 
 

### Import Libraries

In [ ]:
%%capture libraries    
import sys
import os
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install mne
!{sys.executable} -m pip install mne-features
import numpy as np
import matplotlib 
import pathlib
import mne
import seaborn as sns
import pandas as pd
from mne.io import concatenate_raws, read_raw_edf
from mne import Epochs, create_info, events_from_annotations
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs,corrmap
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch, tfr_stockwell,tfr_multitaper,tfr_array_morlet,AverageTFR
from scipy import signal, stats
from scipy.integrate import simps
from scipy.signal import welch
from scipy.stats import ttest_ind
matplotlib.use('Qt5Agg') #allow interactive plots
import matplotlib.pyplot as plt
from mne.decoding import GeneralizingEstimator, Scaler,cross_val_multiscore, LinearModel, get_coef, Vectorizer, CSP, SlidingEstimator
from mne.viz import centers_to_edges
%run EEG_functions_1.ipynb import load_data, excl_chan, filter_data, make_epochs, plot_data, epochs_power #upload the notebook with the python functions



### Load the cleaned epoched dataset

This dataset is filtered between 1-30Hz, artifacts correction with ICA and manual rejections of bad epochs (20 epochs rejected). 

Each epoch has a duration of 9.5secs (0.25 - 9.748)

In [ ]:
epochs= mne.read_epochs('epoched_data_280-epo.fif', preload=True)

### Explore the frequence content of our epochs for each imagery condition



In [ ]:
epochs['Relax'].plot_psd(average=True, fmax=70)

In [ ]:
epochs['Push'].plot_psd(average=True, fmax=70)

### Plot the PSD spatial distribution for each task

In [ ]:
#Define Freqeuncy Bands
bands = [(4, 7, 'Theta'), (8, 12, 'Alpha'), (13, 30, 'Beta')]

In [ ]:
epochs['Relax'].plot_psd_topomap(bands=bands, normalization='full', cmap='RdBu_r')

In [ ]:
epochs['Push'].plot_psd_topomap(bands=bands, normalization='full',  cmap='RdBu_r')

### Compute the PSD based on Welch's method

In [ ]:
#concatenate epochs from the channel of interest in order to have 1 dimensional array
#['AF3','F7','F3','FC5','T7','P7','O1','O2','P8','T8','FC6','F4','F8','AF4']; 
 
data_relax_O1=[]
for sample in range (0, len(epochs['Relax'])):
    x=list(epochs['Relax'].get_data()[sample][6][:]) #define your channel, 6 is O1 and 7 is O2 
    data_relax_O1 += x
    
data_relax_O2=[]
for sample in range (0, len(epochs['Relax'])):
    x=list(epochs['Relax'].get_data()[sample][7][:])  
    data_relax_O2 += x
    
#print(len(data_relax_O1)) #the shape is given by n_epochs*n_samples (type: epochs['Relax'].get_data().shape[0] * 2432)
 
data_push_O1=[]
for sample in range (0, len(epochs['Push'])):
    x=list(epochs['Push'].get_data()[sample][6][:])  
    data_push_O1 += x

data_push_O2=[]
for sample in range (0, len(epochs['Push'])):
    x=list(epochs['Push'].get_data()[sample][7][:])  
    data_push_O2 += x
    
#print(len(data_push))#the shape is given by n_epochs*n_samples (type: epochs['Push'].get_data().shape[0] * 2432)


In [ ]:
#Compute Welch's

sfreq=epochs.info['sfreq'] #sampling rate
win = 4 * sfreq # Define window length (4 seconds)

#O1
Fxx_rO1, Pxx_rO1 = welch(data_relax_O1, sfreq, nperseg=win, window='hanning',  detrend="linear")  #relax condition
Fxx_pO1, Pxx_pO1 = welch(data_push_O1, sfreq, nperseg=win, window='hanning',  detrend="linear") #push condition

#O2
Fxx_rO2, Pxx_rO2 = welch(data_relax_O2, sfreq, nperseg=win, window='hanning',  detrend="linear")  #relax condition
Fxx_pO2, Pxx_pO2 = welch(data_push_O2, sfreq, nperseg=win, window='hanning',  detrend="linear") #push condition

#Concatenate values
rO=np.concatenate([Pxx_rO1,Pxx_rO2]) #relax from occipital channels O1 and O2
pO=np.concatenate([Pxx_pO1,Pxx_pO2]) #push from occipital channels O1 and O2

In [ ]:
#Compute a t-test to inspect if there is a statistically significant difference between the two PSDs
stats.ttest_ind(rO, pO)

In [ ]:
#Compute absolute and relative power in the RELAX condition (taking into account the channels previously described)

low, high=8,12 # alpha band limit
#low, high=13,30 # beta band limit

idx_alpha=np.logical_and(Fxx>=low, Fxx<=high) 

#absolute power
freq_res=Fxx_rO1[1]-Fxx_rO1[0]
#absolute power by approximateing the area under the curve
alpha_power=simps(Pxx_rO1[idx_alpha], dx=freq_res)
print('Absolute alpha power: %.15f uV^2' %alpha_power)

#relative power
total_power=simps(Pxx_rO1, dx=freq_res)
alpha_rel_power=np.round(np.divide(alpha_power,total_power),3)
print('Relative alpha power: %.15f uV^2' %alpha_rel_power)
print('Thus, ', alpha_rel_power*100, '% of the total power of the signal is contained in alpha frequency band' )

In [ ]:
# Relax Plot - Alpha Band

%matplotlib inline
plt.plot(Fxx_rO1, Pxx_rO1)
plt.fill_between(Fxx_rO1, Pxx_rO1, where=idx_alpha, color='orange')  
plt.xlim([2, 30])
plt.ylim([0, Pxx.max()*1.1])
plt.xlabel('Frequency (Hz)')
plt.ylabel("Power spectral density")
plt.title("Welch's Periodogram - Relax O1 Alpha Band ")
plt.show()

In [ ]:
#Compute absolute and relative power in the PUSH condition

low, high=8,12 # alpha band limit
#low, high=13,30 # beta band limit

idx_alpha=np.logical_and(Fxx_p>=low, Fxx_p<=high)

#absolute power
freq_res=Fxx_pO1[1]-Fxx_pO1[0]
#absolute power by approximateing the area under the curve
alpha_power=simps(Pxx_pO1[idx_alpha], dx=freq_res)
print('Absolute alpha power: %.15f uV^2' %alpha_power)

#relative power
total_power=simps(Pxx_pO1, dx=freq_res)
alpha_rel_power=np.round(np.divide(alpha_power,total_power),3)
print('Relative alpha power: %.15f uV^2' %alpha_rel_power)
print('Thus, ', alpha_rel_power*100, '% of the total power of the signal is contained in alpha frequency band' )

In [ ]:
# Plot Push - Alpha Band

%matplotlib inline
plt.plot(Fxx_pO1, Pxx_pO1)
plt.fill_between(Fxx_pO1, Pxx_pO1, where=idx_alpha, color='orange')
plt.xlim([2, 30])
plt.ylim([0, Pxx.max()*1.1])
plt.xlabel('Frequency (Hz)')
plt.ylabel("Power spectral density")
plt.title("Welch's Periodogram - Push O1 Alpha Band")
plt.show()

### Compute Time-Frequency Analysis (TFR) on epoched data using Morlet Wavelet Transform and FFT for convolutions 

In [ ]:
freqs = np.logspace(*np.log10([2, 30]), num=40) # define frequencies of interest (log-spaced) 
n_cycles = freqs / 2.  # number of cycle increase linearly per frequency


#Compute power for RELAX condition
power_r = mne.time_frequency.tfr_morlet(epochs['Relax'], freqs=freqs, n_cycles=n_cycles, 
                                           use_fft=True, average=True,
                                           return_itc=False, decim=3, n_jobs=1)

#Compute power for PUSH condition
power_p= mne.time_frequency.tfr_morlet(epochs['Push'], freqs=freqs, n_cycles=n_cycles, 
                                           use_fft=True, average=True,
                                           return_itc=False, decim=3, n_jobs=1)

### Extract the features of interest. Compute  the average across frequencies for each band  (theta, alpha, beta) 

In [ ]:
# Relax Imagery Condition - Alpha Activity

relax_pow_a = [] #store the power average for each channel  

for chan in range(0, len(epochs['Relax'].ch_names)): #for each sample in the range 0-14
    pow_ar = power_r.data[chan][(power_r.freqs>=8) & (power_r.freqs<12)] #select the alpha band freqs range
    pow_avg_ar = np.mean(pow_ar, axis=0) #compute the average
    relax_pow_a.append(pow_avg_ar) 
    

In [ ]:
# Relax Imagery Condition - Beta Activity

relax_pow_b = [] #store the power average for each channel  

for chan in range(0, len(epochs['Relax'].ch_names)): #for each sample in the range 0-14
    pow_br = power_r.data[chan][(power_r.freqs>=13) & (power_r.freqs<30)] #select the beta band freqs range
    pow_avg_br = np.mean(pow_br, axis=0) #compute the average
    relax_pow_b.append(pow_avg_br)  
    
    

In [ ]:
# Relax Imagery Condition - Theta Activity

relax_pow_t = [] #store the power average for each channel  

for chan in range(0, len(epochs['Relax'].ch_names)): #for each sample in the range 0-14
    pow_tr = power_r.data[chan][(power_r.freqs>=4) & (power_r.freqs<7)] #select the beta band freqs range
    pow_avg_tr = np.mean(pow_tr, axis=0) #compute the average
    relax_pow_t.append(pow_avg_tr)  

In [ ]:
# Push Imagery Condition - Alpha Activity

push_pow_a = []

for chan in range(0, len(epochs['Push'].ch_names)):
    pow_ap = power_p.data[chan][(power_p.freqs>=8) & (power_p.freqs<12)] 
    pow_avg_ap = np.mean(pow_ap, axis=0)
    push_pow_a.append(pow_avg_ap)

In [ ]:
# Push Imagery Condition - Beta Activity

push_pow_b = []

for chan in range(0, len(epochs['Push'].ch_names)):
    pow_bp = power_p.data[chan][(power_p.freqs>=13) & (power_p.freqs<30)] 
    pow_avg_bp = np.mean(pow_bp, axis=0)
    push_pow_b.append(pow_avg_bp)

In [ ]:
# Push Imagery Condition - Theta Activity

push_pow_t = []

for chan in range(0, len(epochs['Push'].ch_names)):
    pow_tp = power_p.data[chan][(power_p.freqs>=4) & (power_p.freqs<7)] 
    pow_avg_tp = np.mean(pow_tp, axis=0)
    push_pow_t.append(pow_avg_tp)

In [ ]:
time=1000 * power_r.times #define the time vector in ms
channels=epochs.info['ch_names'] #store the channels' names

In [ ]:
#Save the plots - Remember to create the folder and change the variable "script_dir" 

#Alpha Activity

script_dir = '.../Desktop/plots_/TFR/alpha/' #specify where you want to save the plots

for chan in range(0,len(channels)):
    plt.figure()
    plt.plot(time,relax_pow_a[chan]) #relax task
    plt.plot(time, push_pow_a[chan]) #push task
    plt.legend(["Relax", "Push"]) 
    plt.title('Alpha Band - Channel {} '.format(channels[chan]))
    #plt.show()
    filename = "Figure{}.png".format(chan)
    plt.savefig(script_dir + filename)


In [ ]:
#Beta

script_dir = '.../Desktop/plots_/TFR/beta/'


for chan in range(0,len(channels)):
    plt.figure()
    plt.plot(time,relax_pow_b[chan]) 
    plt.plot(time, push_pow_b[chan]) 
    plt.legend(["Relax", "Push"])
    plt.title('Beta Band - Channel {} '.format(channels[chan]))
    #plt.show()  
    filename = "Figure{}.png".format(chan)
    plt.savefig(script_dir + filename)
    

In [ ]:
#Theta

script_dir = '.../Desktop/plots_/TFR/alpha/'

for chan in range(0,len(channels)):  
    plt.figure()  
    plt.plot(time,relax_pow_t[chan])  
    plt.plot(time, push_pow_t[chan]) 
    plt.legend(["Relax", "Push"])
    plt.title('Theta Band - Channel {} '.format(channels[chan]))
    #plt.show()
    filename = "Figure{}.png".format(chan)
    plt.savefig(script_dir + filename)

### Compute the TFR analysis directly on a numpy array 

In [ ]:
freqs = np.logspace(*np.log10([4, 27]), num=40) # define frequencies of interest (log-spaced)  
n_cycles =freqs / 2. #define number of cycles
sfreq=epochs['Push'].info['sfreq'] #extract the sampling rate

power_relax = tfr_array_morlet(epochs['Relax'].get_data(), sfreq=sfreq, 
                               freqs=freqs, n_cycles=n_cycles, output='avg_power')

power_push = tfr_array_morlet(epochs['Push'].get_data(), sfreq=sfreq,
                         freqs=freqs, n_cycles=n_cycles, output='avg_power')



In [ ]:
# Save the Plots from the Push Imagery Condition

channels=epochs['Push'].info['ch_names']
script_dir = '.../Desktop/plots_/TFR/TFR_array/Push/'

for chan in range(0,len(channels)):
    fig, ax = plt.subplots()
    x, y = centers_to_edges(epochs['Push'].times * 1000, freqs)
    mesh = ax.pcolormesh(x, y, power_push[chan], cmap='RdBu_r')  
    ax.set_title('Push - Channel {}'.format(channels[chan]))
    ax.set(ylim=freqs[[0, -1]], xlabel='Time (ms)', ylabel='Frequency [Hz]')
    fig.colorbar(mesh)
    #plt.show()
    filename = "Figure{}.png".format(chan)
    plt.savefig(script_dir + filename)

In [ ]:
# Save the Plots from the Relax Imagery Condition

channels=epochs['Push'].info['ch_names']
script_dir = '.../Desktop/plots_/TFR/TFR_array/Relax/'

for chan in range(0,len(channels)):
    fig, ax = plt.subplots()
    x, y = centers_to_edges(epochs['Relax'].times * 1000, freqs)
    mesh = ax.pcolormesh(x, y, power_relax[chan], cmap='RdBu_r') #choose the channel
    ax.set_title('Relax - Channel {}'.format(channels[chan]))
    ax.set(ylim=freqs[[0, -1]], xlabel='Time (ms)', ylabel='Frequency [Hz]')
    fig.colorbar(mesh)
    #plt.show()
    filename = "Figure{}.png".format(chan)
    plt.savefig(script_dir + filename)

In [ ]:
#Alternative visualisation
bands = [(4, '4 Hz'),(5, '5 Hz'),(6, '6 Hz'),(7, '7 Hz'), (8, '8 Hz'), (10, '10 Hz'), (12, '12 Hz'), (15, '15 Hz'), (20, '20 Hz'), (10, 20, '10-20 Hz')]
epochs['Push'].plot_psd_topomap(bands=bands, vlim='joint', ch_type='eeg', normalize=False)